In [31]:
%load_ext autoreload
from utils import *
from datasets import load_dataset
#from accelerate import Accelerator
#accelerator = Accelerator()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
%autoreload
path_labels = "../datasets/Norway/train/annotations/xmls/"
path_imgs = "../datasets/Norway/train/images/"

create_metadata(path_imgs, path_labels)

Wrote metadata.jsonl with 2914 samples to /Users/kohmann/Documents/Studie/2022 Høst/Visual_intelligence/roadcrack-detection/model_notebooks


In [33]:
%autoreload
from datasets import Dataset, Image
dataset = Dataset.from_json("metadata.jsonl")
dataset = dataset.cast_column("image", Image())

Using custom data configuration default-a9f09cc8e59b7e3a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /Users/kohmann/.cache/huggingface/datasets/json/default-a9f09cc8e59b7e3a/0.0.0. Subsequent calls will reuse this data.


In [34]:
len(dataset)

2914

In [35]:
from transformers import YolosFeatureExtractor
feature_extractor = YolosFeatureExtractor.from_pretrained('hustvl/yolos-small', size=(800,800)) # , reduce_labels=True

loading configuration file preprocessor_config.json from cache at /Users/kohmann/.cache/huggingface/hub/models--hustvl--yolos-small/snapshots/5f960fd774250e41a01086ccbbf5e44d9d603c14/preprocessor_config.json
Feature extractor YolosFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "YolosFeatureExtractor",
  "format": "coco_detection",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "max_size": 1333,
  "size": [
    800,
    800
  ]
}



In [36]:
def transforms(example_batch):
    images = example_batch["image"]
    ids_ = example_batch["image_id"]
    objects = example_batch["annotations"]
    targets = [
        {"image_id": id_, "annotations": object_} for id_, object_ in zip(ids_, objects)
    ]
    inputs = feature_extractor(images=images, annotations=targets , return_tensors="pt")
    return inputs


In [37]:
ds = dataset.train_test_split(test_size=0.2)
train_ds = ds["train"]
test_ds = ds["test"]
train_ds.set_transform(transforms)
test_ds.set_transform(transforms)

In [38]:
train_ds[0]

{'pixel_values': tensor([[[ 0.1254,  0.1254,  0.1426,  ..., -0.6794, -0.6794, -0.7822],
          [ 0.1426,  0.1426,  0.1426,  ..., -0.6794, -0.6623, -0.7479],
          [ 0.1597,  0.1597,  0.1426,  ..., -0.6794, -0.6794, -0.7650],
          ...,
          [-0.8164, -0.7822, -0.7650,  ..., -0.8335, -1.0733, -1.4329],
          [-0.7993, -0.7822, -0.7822,  ..., -0.9705, -1.2274, -1.5699],
          [-0.7479, -0.7993, -0.8849,  ..., -1.0904, -1.3644, -1.6727]],
 
         [[ 0.5203,  0.5203,  0.5378,  ..., -0.2325, -0.2675, -0.5126],
          [ 0.5378,  0.5378,  0.5378,  ..., -0.2325, -0.2500, -0.4776],
          [ 0.5553,  0.5553,  0.5378,  ..., -0.2325, -0.2675, -0.4951],
          ...,
          [-0.7577, -0.7227, -0.6877,  ..., -0.7052, -0.9328, -1.2654],
          [-0.7577, -0.7402, -0.7227,  ..., -0.8627, -1.0728, -1.4055],
          [-0.7052, -0.7752, -0.8627,  ..., -0.9853, -1.2479, -1.5455]],
 
         [[ 1.0714,  1.0714,  1.0888,  ...,  0.5659,  0.5834,  0.2348],
          [ 

In [39]:
from transformers import YolosForObjectDetection
label2id = {'D00': 0, 'D10': 1, 'D40': 2, 'D20': 3, } # 'pothole': 4
id2label = {"0":'D00', "1":'D10', "2":'D40', "3":'D20'} # "4":'pothole'

model = YolosForObjectDetection.from_pretrained('hustvl/yolos-small',
                                                id2label=id2label,
                                                label2id=label2id,
                                                ignore_mismatched_sizes=True)

loading configuration file config.json from cache at /Users/kohmann/.cache/huggingface/hub/models--hustvl--yolos-small/snapshots/5f960fd774250e41a01086ccbbf5e44d9d603c14/config.json
Model config YolosConfig {
  "architectures": [
    "YolosForObjectDetection"
  ],
  "attention_probs_dropout_prob": 0.0,
  "auxiliary_loss": false,
  "bbox_cost": 5,
  "bbox_loss_coefficient": 5,
  "class_cost": 1,
  "eos_coefficient": 0.1,
  "giou_cost": 2,
  "giou_loss_coefficient": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "id2label": {
    "0": "D00",
    "1": "D10",
    "2": "D40",
    "3": "D20"
  },
  "image_size": [
    512,
    864
  ],
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "D00": 0,
    "D10": 1,
    "D20": 3,
    "D40": 2
  },
  "layer_norm_eps": 1e-12,
  "model_type": "yolos",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_detection_tokens": 100,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": t

In [40]:
import torch
def collate_fn(batch):
    pixel_values = [item["pixel_values"] for item in batch]
    labels = [item["labels"] for item in batch]
    batch = {}
    batch["pixel_values"] = torch.stack(pixel_values)
    batch["labels"] = labels
    return batch

In [41]:
from transformers import TrainingArguments, Trainer

# We need to specify loss

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    evaluation_strategy="steps",
    num_train_epochs=1,
    fp16=False,
    save_steps=100,
    eval_steps=1,
    logging_steps=1,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=collate_fn,
)

results = trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/Users/kohmann/Documents/Studie/2022 Høst/Visual_intelligence/roadcrack-detection/venv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2331
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 1166
  Number of trainable parameters = 30651273


Step,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 583
  Batch size = 8

KeyboardInterrupt



# Inference (does not currently work)

In [ ]:

from transformers import pipeline, ZeroShotObjectDetectionPipeline
object_detector = pipeline('object-detection',model=model ,feature_extractor=feature_extractor)

In [ ]:
import requests
from utils import *
from PIL import Image

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)
#image = dataset[16]["image"]
results = object_detector(image)
plot_results(image, results)

In [ ]:
results

In [ ]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)
#image = dataset[16]["image"]
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = trainer.model(**inputs)


In [ ]:
target_sizes = torch.tensor(image.size)
target_sizes
results = feature_extractor.post_process_object_detection(outputs, threshold=0.5, target_sizes=target_sizes)

In [ ]:
outputs["logits"].shape